In [1]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

import pandas as pd
import numpy as np

import keras
import tensorflow as tf

import random
import math
import re

Using TensorFlow backend.


In [2]:
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [3]:
def get_sequence_of_tokens(text):
    tokenizer.fit_on_texts(text)
    total_words = len(tokenizer.word_index) + 1
    input_sequences = []
    for line in text:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    model.add(LSTM(700, return_sequences=True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(700, return_sequences=True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(700))
    model.add(Dropout(0.2))
    
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

In [4]:
tokenizer = Tokenizer()

In [5]:
text = (open("dylanThomas.txt").read())
text = text.lower()
words = list(filter(None,re.split('(\n)| ', text)))

words_list = [None] * math.floor((len(words) / 2))

for x in range(math.floor((len(words) / 2))):
    ind_start = random.randint(0, len(words)-7)
    ran = random.randint(4, 7)
    words_list[x] = ' '.join(words[ind_start:ind_start+ran])

In [6]:
inp_sequences, total_words = get_sequence_of_tokens(words_list)

In [7]:
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [8]:
model = create_model(max_sequence_len, total_words)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 8, 10)             50680     
_________________________________________________________________
lstm_1 (LSTM)                (None, 700)               1990800   
_________________________________________________________________
dropout_1 (Dropout)          (None, 700)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5068)              3552668   
Total params: 5,594,148
Trainable params: 5,594,148
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(predictors, label, epochs=200, batch_size=1000)

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
58865/58865 [==============================] - 6s 98us/step - loss: 6.9845
Epoch 2/200
58865/58865 [==============================] - 4s 69us/step - loss: 6.5056
Epoch 3/200
58865/58865 [==============================] - 4s 70us/step - loss: 6.4275
Epoch 4/200
58865/58865 [==============================] - 4s 71us/step - loss: 6.3329
Epoch 5/200
58865/58865 [==============================] - 4s 71us/step - loss: 6.2573
Epoch 6/200
58865/58865 [==============================] - 4s 71us/step - loss: 6.1705
Epoch 7/200
58865/58865 [==============================] - 4s 72us/step - loss: 6.0615
Epoch 8/200
58865/58865 [==============================] - 4s 71us/step - loss: 5.9514
Epoch 9/200
58865/58865 [==============================] - 4s 71us/step - loss: 5.8409
Epoch 10/200
58865/58865 [==============================] - 4s 72us/step - loss: 5.7319
Epoch 11/200
58865/58865 [==============================] - 4s 72us/step - loss: 

58865/58865 [==============================] - 4s 73us/step - loss: 2.0644
Epoch 92/200
58865/58865 [==============================] - 4s 72us/step - loss: 2.0506
Epoch 93/200
58865/58865 [==============================] - 4s 72us/step - loss: 2.0331
Epoch 94/200
58865/58865 [==============================] - 4s 72us/step - loss: 2.0152
Epoch 95/200
58865/58865 [==============================] - 4s 72us/step - loss: 2.0007
Epoch 96/200
58865/58865 [==============================] - 4s 72us/step - loss: 1.9792
Epoch 97/200
58865/58865 [==============================] - 4s 72us/step - loss: 1.9624
Epoch 98/200
58865/58865 [==============================] - 4s 72us/step - loss: 1.9512
Epoch 99/200
58865/58865 [==============================] - 4s 72us/step - loss: 1.9335
Epoch 100/200
58865/58865 [==============================] - 4s 72us/step - loss: 1.9193
Epoch 101/200
58865/58865 [==============================] - 4s 72us/step - loss: 1.9036
Epoch 102/200
58865/58865 [================

58865/58865 [==============================] - 4s 71us/step - loss: 1.2325
Epoch 184/200
58865/58865 [==============================] - 4s 71us/step - loss: 1.2306
Epoch 185/200
58865/58865 [==============================] - 4s 71us/step - loss: 1.2293
Epoch 186/200
58865/58865 [==============================] - 4s 71us/step - loss: 1.2217
Epoch 187/200
58865/58865 [==============================] - 4s 73us/step - loss: 1.2245
Epoch 188/200
58865/58865 [==============================] - 4s 73us/step - loss: 1.2150
Epoch 189/200
58865/58865 [==============================] - 4s 74us/step - loss: 1.2131
Epoch 190/200
58865/58865 [==============================] - 4s 75us/step - loss: 1.2128
Epoch 191/200
58865/58865 [==============================] - 4s 74us/step - loss: 1.2070
Epoch 192/200
58865/58865 [==============================] - 4s 73us/step - loss: 1.2020
Epoch 193/200
58865/58865 [==============================] - 4s 74us/step - loss: 1.1974
Epoch 194/200
58865/58865 [========

In [10]:
model.save_weights('dylanThomasAdam.h5')

In [16]:
generate_text("Poetry", 25, model, max_sequence_len).lower()

'poetry the sun and daughters no longer grieved the sea is cannot of the heart of the heart with a passage of psalms and trees in'